# Always On

## Parámetros

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/AlwaysOn/data'
FILE_NAME = 'op.csv'

## Código

In [22]:
from pathlib import Path
from pandas import DataFrame, read_csv

class AlwaysOn:
    def __init__(self, base_dir, file_name) -> None:
        self.base_dir = Path(base_dir)
        self.file_name = file_name
        self.file_path = self.base_dir.joinpath(self.file_name)
        if not self.file_path.is_file():
            print(f'Debería existir un archivo llamado {self.file_name} en el directorio:\n{self.base_dir}\n\nAgrega el archivo e intenta de nuevo!')
    
    def get_file(self, sample_size: int=1000) -> None:
        try: self.df = read_csv(self.file_path).sample(sample_size)
        except: print(f'No se pudo importar el archivo {self.file_name} desde el directorio:\n{self.base_dir}')
    
    def top_n(self,  group_col: str, count_col: list, n: int) -> DataFrame:
        top = self.df.reset_index().pivot_table(index=[group_col,count_col], values='index', aggfunc='count').reset_index()

        # total = self.df.reset_index().pivot_table(index=group_col, values='index', aggfunc='count').reset_index().rename({'index':'total'}, axis=1)
        # top = top.merge(total, on=group_col)
        # top['index'] /= top['total']

        top[f'rank_{count_col}'] = top.groupby(group_col)['index'].rank(method="first", ascending=False).to_frame()
        top = top[top[f'rank_{count_col}']<=n].sort_values([group_col,f'rank_{count_col}']).drop('index', axis=1)
        return top

    def full_pipeline(self, product_col: str='product') -> DataFrame:
        self.get_file()
        top_categories = self.top_n(group_col='store_category', count_col='product_category', n=5)
        self.df['first_word'] = self.df[product_col].map(lambda x: str(x).split()[0])
        top_products = self.top_n(group_col='store_category', count_col='first_word', n=20)



ao = AlwaysOn(BASE_DIR, FILE_NAME)
df = ao.full_pipeline()

,store_category,product_category,rank_product_category
0,"Carnes, pescados y mariscos",Pavo,1.0
1,"Carnes, pescados y mariscos",Verduras frescas,2.0
2,Chocolates y dulces,Regalos,1.0
3,Conveniencia,Agua,1.0
4,Conveniencia,Pan,2.0
5,Conveniencia,Papas fritas y snacks,3.0
6,Conveniencia,Pilas y baterías,4.0
8,Farmacia,Antibióticos,1.0
7,Farmacia,Analgésicos y desinflamatorios,2.0
9,Farmacia,Cardiovascular,3.0
